In [1]:
pip install accelerate transformers SentencePiece  bitsandbytes einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.6 MB/s eta 0:00:00


In [4]:
import torch
import transformers

config = transformers.AutoConfig.from_pretrained(
  'mosaicml/mpt-7b-instruct',
  trust_remote_code=True
)
config.update({"max_seq_len": 4096})

model = transformers.AutoModelForCausalLM.from_pretrained(
  'mosaicml/mpt-7b-instruct',
  config=config,
  trust_remote_code=True,
  torch_dtype=torch.bfloat16,
)
model.to("cuda:0")

OSError: ignored

In [3]:
!nvidia-smi


Mon Jun 12 15:11:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    54W / 400W |  13955MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained('mosaicml/mpt-7b-instruct')


In [5]:
import pandas as pd

df = pd.read_json('dummy.json')

df

,n.documentName,n.pageContent,ID(n)
0,Lead Paint Law: Information for Owners and Sel...,<p>Maryland and federal laws about lead paint ...,100
1,Baltimore City Rent Escrow Law,<p>The purpose of the rent escrow law is to pr...,101
2,Cosigning a Lease,<p>There are times when a tenant needs a cosig...,111
3,When is Refusal-to-Rent Illegal Housing Discri...,<p>People often wrongly assume that any discri...,112
4,Anne Arundel County Rental and Housing Laws,\r\n <h2>Topics on this page:</h2>\r\n \r\n ...,749
...,...,...,...
452,Contesting a Will (Caveat Proceeding),<p>Contesting a will is a formal way of asking...,23350
453,Time limit for filing a will with the Register...,"\r\n \r\n <p>After someone dies, Maryland la...",23351
454,Youth Health Law,<h2>Topics on this page:</h2>\r\n \r\n <ul>\...,23352
455,Introduction to Juvenile Record Expungement,<p>Maryland law permits a person with a juveni...,23957


In [6]:
from bs4 import BeautifulSoup
import re
import string

#removing html code
def remove_html(text):
    soup = BeautifulSoup(text)
    text = soup.get_text()
    text = text.replace("\n", " ")
    text = text.replace("\xa0", " ")
    return text
df["n.pageContent"] = df["n.pageContent"].apply(remove_html)


In [7]:
content = df["n.pageContent"][0]
content

"Maryland and federal laws about lead paint are designed to reduce childhood lead poisoning. This article describes the duties of landlords and property owners of residential housing built before 1978. Information for renters and buyers can be found in another article.   Topics on this page:  Which homes are covered by Maryland’s lead paint law? Use of lead paint Owning and selling an Affected Property Landlord's duties Frequently asked questions for landlords    Which homes are covered by Maryland’s lead paint law? Only pre-1978 residential rental properties/units (“Affected Properties”) are regulated by Maryland’s Reduction of Lead Risk in Housing Act. Owner-occupied properties/units are not regulated. Affected Properties that are Maryland Department of the Environment (MDE) certified lead-free or limited lead-free are exempt from Maryland’s lead paint law. Affected Properties owned or operated by federal, State, or local government or by a public, quasi-public, or municipal corporat

In [22]:
import torch
torch.cuda.empty_cache()

In [9]:
big_content = content[:5000]
check_big_content_len = big_content.split(" ")
len(check_big_content_len)

784

In [10]:
big_content

"Maryland and federal laws about lead paint are designed to reduce childhood lead poisoning. This article describes the duties of landlords and property owners of residential housing built before 1978. Information for renters and buyers can be found in another article.   Topics on this page:  Which homes are covered by Maryland’s lead paint law? Use of lead paint Owning and selling an Affected Property Landlord's duties Frequently asked questions for landlords    Which homes are covered by Maryland’s lead paint law? Only pre-1978 residential rental properties/units (“Affected Properties”) are regulated by Maryland’s Reduction of Lead Risk in Housing Act. Owner-occupied properties/units are not regulated. Affected Properties that are Maryland Department of the Environment (MDE) certified lead-free or limited lead-free are exempt from Maryland’s lead paint law. Affected Properties owned or operated by federal, State, or local government or by a public, quasi-public, or municipal corporat

In [21]:
question = "What are the landlord's duties "
input_text = " Content: " + big_content + " " + question + "Answer is: "

tokenized_example = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(tokenized_example['input_ids'].to('cuda:0'), do_sample=False, top_k=5, top_p=0.95)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print(answer[0].rstrip())

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 6>:6                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py:115 in decorate_context       │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1522 in generate        │
│                                                                                                  │
│   1519 │   │   │   │   )                                                                         │
│   1520 │   │   │                                                                                 │
│   1521 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1522 │   │   │   return self.greedy_search(                                                    │
│   1523 │   │   │   │   input_ids,                                                                │
│   1524 │   │   │   │   logits_processor=logits_processor,                                        │
│   1525 │   │   │   │   stopping_criteria=stopping_criteria,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2336 in greedy_search   │
│                                                                                                  │
│   2333 │   │   │   │   │   break                                                                 │
│   2334 │   │   │                                                                                 │
│   2335 │   │   │   # prepare model inputs                                                        │
│ ❱ 2336 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2337 │   │   │                                                                                 │
│   2338 │   │   │   # forward pass to get next token                                              │
│   2339 │   │   │   outputs = self(                                                               │
│                                                                                                  │
│ /root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/e7119f37956c1a386 │
│ 5da33e25ef5ce9159ff2c16/modeling_mpt.py:281 in prepare_inputs_for_generation                     │
│                                                                                                  │
│   278 │   │   if inputs_embeds is not None:                                                      │
│   279 │   │   │   raise NotImplementedError('inputs_embeds is not implemented for MPT yet')      │
│   280 │   │   attention_mask = kwargs['attention_mask'].bool()                                   │
│ ❱ 281 │   │   if attention_mask[:, -1].sum() != attention_m

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model)

trainable params: 6649286656 || all params: 6649286656 || trainable%: 100.0


In [ ]:
big_content = "My mother is 78 years old. " + content 
check_big_content_len = big_content.split(" ")
len(check_big_content_len)

2687

In [ ]:
torch.cuda.empty_cache()

In [ ]:
question = "How old is my mother? "
input_text = " Content: " + big_content + " " + question + "Answer is: "

tokenized_example = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(tokenized_example['input_ids'].to('cuda:0'),max_new_tokens=150, do_sample=False, top_k=5, top_p=0.95)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print(answer[0].rstrip())
torch.cuda.empty_cache()

         Temporarily relocate all tenants while work is being performed at the Affected Property and obtain a passing modified risk reduction inspection certificate prior to the tenants moving back in.  MDE receives copies of Notices of EBL and monitors the EBL notices. Regarding Notices of Defect, if an owner fails to comply with this notice, a tenant can make a complaint with MDE by calling 410-537-3825.  A tenant will need to provide MDE with a copy of the Notice of Defect and written verification of the owner’s receipt of the Notice of Defect.  Read the Law: Md. Code, Environment §§ 6-819 and 6-821.  Are there penalties for failing to comply with the Maryland Reduction in Lead Risk in Housing Act? Owners that fail to register or renew an Affected Property with MDE are subject to a penalty of up to $20 per day until the Affected Property is registered or renewed. Owners that fail to obtain an applicable passing lead paint risk reduction inspection certificate or distribute the lead-